<a href="https://colab.research.google.com/github/res1988/workings/blob/master/tf2_structured_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#https://www.tensorflow.org/alpha/tutorials/keras/feature_columns
!pip install sklearn

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd

!pip install tensorflow==2.0.0-alpha0
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split


In [3]:
#pandsa to create a dataframe
URL = 'https://storage.googleapis.com/applied-dl/heart.csv'
dataframe = pd.read_csv(URL)
dataframe.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [4]:
train, test  = train_test_split(dataframe, test_size = 0.2)
train, val = train_test_split(train, test_size = 0.2)
print(len(train), " train examples")
print(len(val), " validation examples")
print(len(test), " test examples")


193  train examples
49  validation examples
61  test examples


In [0]:
#utility to create a tf.data dataset from a Pandas Dataframe

def df_to_dataset(dataframe,  shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop("target")
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size = len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [0]:
batch_size = 5 # small used for demo...
train_ds = df_to_dataset(train, batch_size = batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)



In [9]:
#understand what you have just done...
for feature_batch, label_batch in train_ds.take(1):
  print("Every feature: ", list(feature_batch.keys()))
  print("A batch of ages: ", feature_batch['age'])
  print("A batch of targets: ", label_batch)

Every feature:  ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']
A batch of ages:  tf.Tensor([69 54 65 70 65], shape=(5,), dtype=int32)
A batch of targets:  tf.Tensor([1 0 0 0 0], shape=(5,), dtype=int32)


In [0]:
#use a batch to demonstrate feature columns
example_batch = next(iter(train_ds))[0]

#utility method to create feature column
# an transform a batch of data
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

In [12]:
#numeric columns
age = feature_column.numeric_column("age")
demo(age)

W0514 03:27:15.535694 139622801385344 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/feature_column/feature_column_v2.py:2758: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.


[[69.]
 [54.]
 [65.]
 [70.]
 [65.]]


In [13]:
#bucketized columns
age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
demo(age_buckets)

W0514 03:29:03.117981 139622801385344 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/feature_column/feature_column_v2.py:2902: to_int64 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.


[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [14]:
#categorical columns
thal = feature_column.categorical_column_with_vocabulary_list("thal", ["fixed", "normal", "reversible"])
thal_one_hot = feature_column.indicator_column(thal)
demo(thal_one_hot)

W0514 03:30:48.057749 139622801385344 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/feature_column/feature_column_v2.py:4307: IndicatorColumn._variable_shape (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
W0514 03:30:48.061380 139622801385344 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/feature_column/feature_column_v2.py:4362: VocabularyListCategoricalColumn._num_buckets (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


[[0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [15]:
#embedding columns if features are too sparse - quite easy to create...
thal_embedding = feature_column.embedding_column(thal, dimension=8)
demo(thal_embedding)

[[-1.54648498e-01  3.41571152e-01  2.07995087e-01  1.95993595e-02
   1.08042836e-01  2.25739315e-01  6.79666176e-02 -3.92386764e-01]
 [-1.54648498e-01  3.41571152e-01  2.07995087e-01  1.95993595e-02
   1.08042836e-01  2.25739315e-01  6.79666176e-02 -3.92386764e-01]
 [-1.57970563e-01 -4.40186024e-01 -5.03359318e-01  3.72690827e-01
  -3.67936134e-01  2.04672262e-01 -2.41388753e-02 -5.26364529e-05]
 [-1.57970563e-01 -4.40186024e-01 -5.03359318e-01  3.72690827e-01
  -3.67936134e-01  2.04672262e-01 -2.41388753e-02 -5.26364529e-05]
 [-1.54648498e-01  3.41571152e-01  2.07995087e-01  1.95993595e-02
   1.08042836e-01  2.25739315e-01  6.79666176e-02 -3.92386764e-01]]


In [0]:
#engineer different features
feature_columns = []

# numeric cols
for header in ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'slope', 'ca']:
  feature_columns.append(feature_column.numeric_column(header))

# bucketized cols
age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
feature_columns.append(age_buckets)

# indicator cols
thal = feature_column.categorical_column_with_vocabulary_list(
      'thal', ['fixed', 'normal', 'reversible'])
thal_one_hot = feature_column.indicator_column(thal)
feature_columns.append(thal_one_hot)

# embedding cols
thal_embedding = feature_column.embedding_column(thal, dimension=8)
feature_columns.append(thal_embedding)

# crossed cols
crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
crossed_feature = feature_column.indicator_column(crossed_feature)
feature_columns.append(crossed_feature)


In [0]:
#feature layer
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [0]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [20]:
model = tf.keras.Sequential([
    feature_layer,
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer = 'adam',
             loss='binary_crossentropy',
             metrics = ['accuracy'])

model.fit(train_ds, validation_data= val_ds, epochs=5)

W0514 03:39:14.250859 139622801385344 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/feature_column/feature_column_v2.py:4362: CrossedColumn._num_buckets (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


Epoch 1/5
7/7 [==============================] - 1s 106ms/step - loss: 1.5321 - accuracy: 0.6417 - val_loss: 0.5033 - val_accuracy: 0.7959
Epoch 2/5
7/7 [==============================] - 0s 32ms/step - loss: 0.6019 - accuracy: 0.7109 - val_loss: 0.5330 - val_accuracy: 0.7755
Epoch 3/5
7/7 [==============================] - 0s 33ms/step - loss: 0.5793 - accuracy: 0.6775 - val_loss: 0.6151 - val_accuracy: 0.6735
Epoch 4/5
7/7 [==============================] - 0s 31ms/step - loss: 0.5338 - accuracy: 0.6910 - val_loss: 0.4483 - val_accuracy: 0.8163
Epoch 5/5
7/7 [==============================] - 0s 30ms/step - loss: 0.4945 - accuracy: 0.7201 - val_loss: 0.4394 - val_accuracy: 0.8163


In [21]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)


2/2 [==============================] - 0s 18ms/step - loss: 0.4783 - accuracy: 0.8033
Accuracy 0.8032787
